In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
((sqrt(2)-I)^3+(sqrt(2)-I)).expand()

-6*I

In [3]:
(sqrt(2)-I).minpoly('xx')

xx^4 - 2*xx^2 + 9

In [4]:
var('xx')
Q.<rad2_minus_i> = NumberField(xx^4 - 2*xx^2 + 9)

In [5]:
ii = ((rad2_minus_i)^3+rad2_minus_i)/(-6)

In [6]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [7]:
e_sost = {e: (-4*ii)*c + d + (-2*ii + 1)*f}
d_sost = {d: ((-64*ii - 48)*c^2 + (64*ii + 128)*c*f + (64*ii + 48)*f^2)/((32*ii - 16)*c + (-32*ii + 16)*f)}

In [8]:
SE1_1 = SE1.subs(e_sost).subs(d_sost)

In [9]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 6.39 s, sys: 149 ms, total: 6.53 s
Wall time: 35.4 s


576

In [10]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 7.19 s, sys: 140 ms, total: 7.33 s
Wall time: 18.8 s


4

In [11]:
%time conds = SE1_1.find_conditions_for_subfamilies(adm_SE1_1, simm_SE1_1)

CPU times: user 7.25 s, sys: 177 ms, total: 7.43 s
Wall time: 2min 43s


In [12]:
conds

[([Ideal (6*c + (-rad2_minus_i^3 - rad2_minus_i^2 + rad2_minus_i + 3)*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in rad2_minus_i with defining polynomial xx^4 - 2*xx^2 + 9,
   Ideal (18*c + (-rad2_minus_i^3 + 3*rad2_minus_i^2 - 7*rad2_minus_i + 3)*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in rad2_minus_i with defining polynomial xx^4 - 2*xx^2 + 9],
  ('E1', 'F14', 'G1', 'F15', 'F35')),
 ([Ideal (9*c + (-rad2_minus_i^3 + 2*rad2_minus_i)*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in rad2_minus_i with defining polynomial xx^4 - 2*xx^2 + 9,
   Ideal (3*c + (-rad2_minus_i)*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in rad2_minus_i with defining polynomial xx^4 - 2*xx^2 + 9],
  ('G4', 'E1', 'G5', 'F45', 'G2')),
 ([Ideal (6*c + (rad2_minus_i^3 - 3*rad2_minus_i^2 + rad2_minus_i + 3)*f) of Multivariate Polynomial Rin

In [13]:
prim_decs = [el[0] for el in conds]
ideals = [ideal for prim_dec in prim_decs for ideal in prim_dec]

In [14]:
sosts = [solve_linear_system(ideal.gens(), [c], [f]) for ideal in ideals]

In [15]:
cubics = [SE1_1.subs({c:sost[0], f:sost[1]}) for sost in list(set(sosts))]

In [16]:
main_cubic_index = sosts.index(((rad2_minus_i)*f, 3*f)) #we can choose any cubic

In [17]:
L_sets = []
for L_set in [get_permuted_L_set(perm) for perm in SE1_1.find_admissible_permutations()]:
    if L_set not in L_sets:
        L_sets.append(L_set)

In [18]:
%%time
main_cubic = cubics[main_cubic_index]
equi = [_ for i in range(len(cubics))]
equi[main_cubic_index] = main_cubic.L_set_base
for cubic in cubics[0:main_cubic_index]+cubics[main_cubic_index+1:]:
    for L_set in L_sets:
        proj = cubic.find_projectivity(cubic.L_set_base, L_set)[0]
        eqn = cubic.eqn.subs(change_coordinates(proj))
        if main_cubic.are_cubics_same(eqn):
            equi[cubics.index(cubic)] = L_set
            break

CPU times: user 2min 9s, sys: 271 ms, total: 2min 9s
Wall time: 2min 9s


In [19]:
equi

[('E1', 'G4', 'F46', 'G6', 'F45'),
 ('E1', 'F14', 'F36', 'G3', 'F56'),
 ('E1', 'G4', 'E2', 'F12', 'E3'),
 ('E1', 'G4', 'E2', 'G3', 'E3'),
 ('F14', 'E1', 'G2', 'E4', 'F15'),
 ('F14', 'E1', 'F13', 'F26', 'F16'),
 ('F14', 'E1', 'G2', 'F25', 'F15'),
 ('E1', 'F14', 'F36', 'F15', 'F56'),
 ('F14', 'E1', 'F13', 'F56', 'F16'),
 ('E1', 'F14', 'F35', 'F12', 'E4'),
 ('E1', 'F14', 'F35', 'F16', 'E4'),
 ('E1', 'G4', 'E6', 'G5', 'E5')]

In [20]:
SE1_2 = main_cubic
SE1_2

x^2*y - x*y^2 + (-1/51*rad2_minus_i^3 - 2/51*rad2_minus_i^2 - 19/51*rad2_minus_i - 38/51)*x^2*z + (-1/306*rad2_minus_i^3 - 2/17*rad2_minus_i^2 + 83/306*rad2_minus_i + 13/17)*x*y*z + (5/102*rad2_minus_i^3 + 5/51*rad2_minus_i^2 - 7/102*rad2_minus_i - 7/51)*x*z^2 + (7/153*rad2_minus_i^3 - 1/51*rad2_minus_i^2 + 31/153*rad2_minus_i - 19/51)*x*y*t + (-7/102*rad2_minus_i^3 - 7/51*rad2_minus_i^2 - 31/102*rad2_minus_i + 20/51)*y^2*t + (7/306*rad2_minus_i^3 + 8/51*rad2_minus_i^2 + 31/306*rad2_minus_i - 1/51)*y*z*t + (-4/153*rad2_minus_i^3 + 1/17*rad2_minus_i^2 + 26/153*rad2_minus_i + 2/17)*y*t^2

In [21]:
%time adm_SE1_2 = SE1_2.find_admissible_projectivities()
len(adm_SE1_2)

CPU times: user 3.1 s, sys: 128 ms, total: 3.23 s
Wall time: 15.5 s


576

In [22]:
%time simm_SE1_2 = SE1_2.find_simmetries(adm_SE1_2)
len(simm_SE1_2)

CPU times: user 2.89 s, sys: 124 ms, total: 3.02 s
Wall time: 3.43 s


8

In [24]:
lines_perms = [Permutation(SE1_2.apply_proj_to_lines(simm[0])).to_permutation_group_element() for simm in simm_SE1_2]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(8, 'C8')

In [25]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()]

[('F14', 'G4', 'E5', 'G1', 'F46', 'E3')]

In [36]:
perm = lines_perms_group.gens_small()[0]
perm

(1,15)(2,5,3,26,19,25,22,6)(4,17,21,9,7,12,23,14)(8,18,16,24,13,27,11,20)

In [37]:
get_permuted_extended_L_set(perm)

('F14', 'G4', 'E5', 'G1', 'F46', 'E3')

In [43]:
%time get_perm_from_extended_L_set(get_permuted_extended_L_set(perm))

CPU times: user 209 ms, sys: 12.1 ms, total: 221 ms
Wall time: 232 ms


(1,15)(2,5,3,26,19,25,22,6)(4,17,21,9,7,12,23,14)(8,18,16,24,13,27,11,20)